# DATA EXPLORATION
Dataset: https://www.kaggle.com/datasets/ahmedabbas757/coffee-sales/data

## **1. Import thư viện:**

In [1]:
import pandas as pd
import numpy as np

## **2. Tiền xử lý:**
### **Thu thập dữ liệu:**

In [2]:
data = pd.read_excel("./data/Coffee Shop Sales.xlsx")
data = pd.DataFrame(data)
data.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


---

### **Khám phá dữ liệu**
#### **- Tập dữ liệu có bao nhiêu dòng? Bao nhiêu cột?**

In [3]:
shape = data.shape
shape

(149116, 11)

#### **- Ý nghĩa của mỗi dòng trong tập dữ liệu?**

Mỗi dòng trong tập dữ liệu thể hiện một thông tin giao dịch có ngày giờ cụ thể tạo một cửa hàng cà phê ở NYC.

#### **- Ý nghĩa của mỗi cột trong tập dữ liệu?**

Dưới đây là ý nghĩa của từng cột: 

| Giá trị cột          | Mô tả                                                 |
|----------------------|-------------------------------------------------------|
| `transaction_id`     | mã giao dịch                                          |
| `transaction_date`   | ngày giao dịch (MM/DD/YY)                             |
| `transaction_time`   | thời gian giao dịch trong ngày (HH:MM:SS)             |
| `transaction_qty`    | số lượng sản phẩm bán trong một giao dịch             |
| `store_id`           | mã của hàng thực hiện giao dịch                       |
| `store_location`     | địa chỉ cửa hàng thực hiện giao dịch                  |
| `product_id`         | mã sản phẩm được bán                                  |
| `unit_price`         | đơn giá của sản phẩm được bán                         |
| `product_category`   | mô tả phân loại sản phẩm                              |
| `product_type`       | mô tả loại sản phẩm                                   |
| `product_detail`     | chi tiết sản phẩm                                     |


#### **- Tập dữ liệu có tồn tại những hàng trùng nhau không?**

In [4]:
duplicates = data[data.duplicated(keep = "first")]
print(f'Raw data have {len(duplicates)} duplicated lines.')

Raw data have 0 duplicated lines.


#### **- Kiểu dữ liệu của mỗi cột là gì? Có hợp lý với ý nghĩa của cột không?**

In [5]:
data.dtypes

transaction_id               int64
transaction_date    datetime64[ns]
transaction_time            object
transaction_qty              int64
store_id                     int64
store_location              object
product_id                   int64
unit_price                 float64
product_category            object
product_type                object
product_detail              object
dtype: object

Các cột dữ liệu: `transaction_id`, `store_id`, `product_id` đang được lưu trữ với kiểu dữ liệu số. Tuy nhiên, các thuộc tính này lại mang ý nghĩa về sự phân loại nên chuyển các thuộc tính trên thành kiểu dữ liệu chuỗi.

In [6]:
data['transaction_id'] = data['transaction_id'].astype(str)
data['store_id'] = data['store_id'].astype(str)
data['product_id'] = data['product_id'].astype(str)

In [7]:
data.dtypes

transaction_id              object
transaction_date    datetime64[ns]
transaction_time            object
transaction_qty              int64
store_id                    object
store_location              object
product_id                  object
unit_price                 float64
product_category            object
product_type                object
product_detail              object
dtype: object

#### **- Sự phân bố dữ liệu?**
**Đối với dữ liệu số**

Thực hiện một số thống kê cơ bản gồm: tỉ lệ mất dữ liệu `missing_ratio (%)`, giá trị nhỏ nhất `min`, các điểm phân vị: `lower quartile` và `upper quartile`, giá trị trung bình `median`, giá trị lớn nhất `max`.

In [8]:
numeric_cols = ['transaction_qty', 'unit_price']
numeric_data = data[numeric_cols]

idx = ['missing_ratio', 'min', 'lower_quartile', 'median', 'upper_quartile', 'max']
col_info_dict = {key: [
                        round(np.sum(np.isnan(numeric_data[key].values))/numeric_data.shape[0] * 100, 1), 
                        round(numeric_data[key].min(), 1),
                        round(numeric_data[key].quantile(0.25), 1),
                        round(numeric_data[key].median(), 1),
                        round(numeric_data[key].quantile(0.75), 1),
                        round(numeric_data[key].max(), 1)
                    ] for key in numeric_cols}
num_col_info_df = pd.DataFrame(col_info_dict, index=idx)
num_col_info_df

,transaction_qty,unit_price
missing_ratio,0.0,0.0
min,1.0,0.8
lower_quartile,1.0,2.5
median,1.0,3.0
upper_quartile,2.0,3.8
max,8.0,45.0


Tiếp theo, xử lý dữ liệu trống trong các cột dữ liệu số

In [9]:
dict(num_col_info_df.iloc[0])

{'transaction_qty': 0.0, 'unit_price': 0.0}

*Ta có thể thấy không tồn tại ô dữ liệu trống trong các cột dữ liệu số của tập dataset!*

**Đối với dữ liệu dạng phân loại**

Tính toán tần suất xuất hiện của các cột và in ra *n* giá trị có tần suất xuất hiện nhiều nhất của mỗi cột

In [10]:
categorical_cols = data.columns[~data.columns.isin(['transaction_qty', 'unit_price'])]

def display_top_categories(df, column, top_n=10):
    top_values = df[column].value_counts()
    
    check = top_n
    top_n = min(top_n, top_values.shape[0])
    
    first_n_values = top_values.head(top_n)
    
    total_count = df.shape[0]
    
    percentages = (top_values / total_count * 100).round(5)
    first_n_percentages = percentages.head(top_n)

    print(f"Tỉ lệ phần trăm các ô trống trong cột '{column}' (missing_ratio): {df[column].isna().sum()}%")
    print(f"Các giá trị trong cột '{column}' với tần xuất và tỉ lệ phần trăm:")

    for value, count, pct in zip(first_n_values.index, first_n_values.values, percentages):
        print(f"{value}: {count} ({pct}%)")

    if check == top_n:
        print("...")
    print("\n" + "-"*40 + "\n")

for col in categorical_cols:
    display_top_categories(data, col)


Tỉ lệ phần trăm các ô trống trong cột 'transaction_id' (missing_ratio): 0%
Các giá trị trong cột 'transaction_id' với tần xuất và tỉ lệ phần trăm:
1: 1 (0.00067%)
99641: 1 (0.00067%)
99661: 1 (0.00067%)
99662: 1 (0.00067%)
99663: 1 (0.00067%)
99664: 1 (0.00067%)
99665: 1 (0.00067%)
99666: 1 (0.00067%)
99667: 1 (0.00067%)
99668: 1 (0.00067%)
...

----------------------------------------

Tỉ lệ phần trăm các ô trống trong cột 'transaction_date' (missing_ratio): 0%
Các giá trị trong cột 'transaction_date' với tần xuất và tỉ lệ phần trăm:
2023-06-19 00:00:00: 1343 (0.90064%)
2023-06-16 00:00:00: 1331 (0.89259%)
2023-06-18 00:00:00: 1290 (0.8651%)
2023-06-13 00:00:00: 1281 (0.85906%)
2023-06-27 00:00:00: 1277 (0.85638%)
2023-06-08 00:00:00: 1269 (0.85102%)
2023-06-15 00:00:00: 1237 (0.82956%)
2023-06-09 00:00:00: 1224 (0.82084%)
2023-05-16 00:00:00: 1219 (0.81748%)
2023-06-24 00:00:00: 1214 (0.81413%)
...

----------------------------------------

Tỉ lệ phần trăm các ô trống trong cột 'tran

*Ta có thể thấy sự phân bố dữ liệu rõ ràng và không tồn tại bất kỳ ô trống nào trong các cột dữ liệu kiểu phân loại của dataset này!*

#### **- Một vài tiền xử lý khác**

- Cột dữ liệu `transaction_date` được tách thành các cột lẻ `year`, `month`, `date`, `day` để dễ dàng xử lý cho sau này.

In [11]:
data['year'] = data['transaction_date'].dt.year
data['month'] = data['transaction_date'].dt.month
data['date'] = data['transaction_date'].dt.day
data['day'] = data['transaction_date'].dt.day_name()

- Tương tự đối với cột `transaction_time` sẽ được tách ra thành 2 cột `hour` và `time` (lưu lần lượt mốc thời gian xảy ra giao dịch, và buổi thực hiện giao dịch).

In [12]:
data['hour'] = data['transaction_time'].apply(lambda x: x.hour)
data['time'] = pd.cut(data['hour'], bins = [6, 12, 17, 20], 
                      include_lowest=True, labels = ['morning', 'afternoon', 'evening'])

In [13]:
data.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,year,month,date,day,hour,time
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg,2023,1,1,Sunday,7,morning
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,2023,1,1,Sunday,7,morning
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,2023,1,1,Sunday,7,morning
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm,2023,1,1,Sunday,7,morning
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,2023,1,1,Sunday,7,morning


- Thêm cột `Total Revenue` để tính tổng doanh thu của một giao dịch

Ta nhận thấy, tổng doanh thu của một giao dịch bằng *tổng số lượng sản phẩm đã bán* * *đơn giá 1 sản phẩm*

In [14]:
data['total_revenue'] = data['transaction_qty'] * data['unit_price']
data

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,year,month,date,day,hour,time,total_revenue
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.00,Coffee,Gourmet brewed coffee,Ethiopia Rg,2023,1,1,Sunday,7,morning,6.00
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,2023,1,1,Sunday,7,morning,6.20
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,2023,1,1,Sunday,7,morning,9.00
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.00,Coffee,Drip coffee,Our Old Time Diner Blend Sm,2023,1,1,Sunday,7,morning,2.00
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,2023,1,1,Sunday,7,morning,6.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149111,149452,2023-06-30,20:18:41,2,8,Hell's Kitchen,44,2.50,Tea,Brewed herbal tea,Peppermint Rg,2023,6,30,Friday,20,evening,5.00
149112,149453,2023-06-30,20:25:10,2,8,Hell's Kitchen,49,3.00,Tea,Brewed Black tea,English Breakfast Lg,2023,6,30,Friday,20,evening,6.00
149113,149454,2023-06-30,20:31:34,1,8,Hell's Kitchen,45,3.00,Tea,Brewed herbal tea,Peppermint Lg,2023,6,30,Friday,20,evening,3.00
149114,149455,2023-06-30,20:57:19,1,8,Hell's Kitchen,40,3.75,Coffee,Barista Espresso,Cappuccino,2023,6,30,Friday,20,evening,3.75


- Xóa các cột không cần thiết cho việc phân tích dataset sau này (`product_detail`)

In [15]:
del data['product_detail']
data

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,year,month,date,day,hour,time,total_revenue
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.00,Coffee,Gourmet brewed coffee,2023,1,1,Sunday,7,morning,6.00
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,2023,1,1,Sunday,7,morning,6.20
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.50,Drinking Chocolate,Hot chocolate,2023,1,1,Sunday,7,morning,9.00
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.00,Coffee,Drip coffee,2023,1,1,Sunday,7,morning,2.00
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,2023,1,1,Sunday,7,morning,6.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149111,149452,2023-06-30,20:18:41,2,8,Hell's Kitchen,44,2.50,Tea,Brewed herbal tea,2023,6,30,Friday,20,evening,5.00
149112,149453,2023-06-30,20:25:10,2,8,Hell's Kitchen,49,3.00,Tea,Brewed Black tea,2023,6,30,Friday,20,evening,6.00
149113,149454,2023-06-30,20:31:34,1,8,Hell's Kitchen,45,3.00,Tea,Brewed herbal tea,2023,6,30,Friday,20,evening,3.00
149114,149455,2023-06-30,20:57:19,1,8,Hell's Kitchen,40,3.75,Coffee,Barista Espresso,2023,6,30,Friday,20,evening,3.75


#### **- Ghi ra file tập dữ liệu đã qua tiền xử lý**

In [16]:
path = "./data/processed_data.csv"
data.to_csv(path, index = False)